In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data
df = pd.read_csv("personalized_social_scheduler_10k.csv")

# Compute Happiness Score (Ensure it remains above 60%)
df["Happiness_Score"] = (df["Mood_After"] - df["Mood_Before"]) * 10 + 60
df["Happiness_Score"] = df["Happiness_Score"].clip(lower=60, upper=100)

# Compute Social Interaction Score (Normalize & Prevent Excess)
df["Social_Interaction_Score"] = (df["Interaction_Frequency_Score"] + df["Social_Preference_Score"]) / 2
df["Social_Interaction_Score"] = df["Social_Interaction_Score"].clip(upper=8)

# Encode categorical features
label_encoders = {}
for col in ['Day_of_Week', 'Friend_ID', 'Preferred_Time_Slot']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Scale numerical values
scaler = MinMaxScaler()
numerical_cols = ['Happiness_Score', 'Social_Interaction_Score']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Define features & target
X = df[['Day_of_Week', 'Friend_ID', 'Preferred_Time_Slot', 'Social_Interaction_Score']]
y = df['Happiness_Score']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape for LSTM input
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

# Build LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Normalize output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Test with a sample input
sample_input = np.array([[4, 12, 2, 5]])  # Random values for testing
sample_input = np.expand_dims(sample_input, axis=1)
predicted_happiness = model.predict(sample_input)


# Decode categorical values back to original labels
decoded_day = label_encoders['Day_of_Week'].inverse_transform([4])[0]
decoded_friend = label_encoders['Friend_ID'].inverse_transform([12])[0]
decoded_time_slot = label_encoders['Preferred_Time_Slot'].inverse_transform([2])[0]

# Print meaningful results
print(f"Predicted Happiness Score: {predicted_happiness}")
print(f"Day of the Week: {decoded_day}")
print(f"Friend Interacted With: {decoded_friend}")
print(f"Preferred Time Slot: {decoded_time_slot}")
# Print results
print("Predicted Happiness Score:", predicted_happiness[0][0])

# Evaluate model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(f"Model Performance - MAE: {mae:.4f}, MSE: {mse:.4f}")


c:\Users\Dhruv\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.1629 - mae: 0.3624 - val_loss: 0.1556 - val_mae: 0.3576
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1589 - mae: 0.3596 - val_loss: 0.1563 - val_mae: 0.3589
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1593 - mae: 0.3606 - val_loss: 0.1551 - val_mae: 0.3559
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1565 - mae: 0.3565 - val_loss: 0.1567 - val_mae: 0.3598
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1591 - mae: 0.3602 - val_loss: 0.1549 - val_mae: 0.3559
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1540 - mae: 0.3537 - val_loss: 0.1557 - val_mae: 0.3582
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1590 - mae: 0.3600 - val_loss: 0.1550 - val_mae: 0.3555
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1599 - mae: 0.3608 - val_loss: 0.1548 - val_mae: 0.3553
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - lo